<a href="https://colab.research.google.com/github/vuduclyunitn/learning_python/blob/master/T%C3%ACm_hi%E1%BB%83u_v%E1%BB%81_c%C3%A1c_trang_tr%C3%AD_h%C3%A0m_(Function_decorators)_v%C3%A0_c%C3%A1c_k%E1%BA%BFt_th%C3%BAc_h%C3%A0m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Các trang trí hàm cho phép ta đánh dấu các hàm trong mã nguồn để nâng cao khả năng của chúng. Đây là một công cụ mạnh mẽ, nhưng để giỏi về nó yêu cầu ta phải hiểu về các kết thúc (closures).

Một trong số các từ khoá được để dành mới nhất trong Python là ```nonlocal```, từ khoá được giới thiệu trong Python 3.0. Nếu bạn muốn triển khai các decorators của riêng mình, bạn phải biết các closures bên trong, khi đó bạn cần tới ```nonlocal```.

Closures còn rất quan trọng trong việc lập trình bất đồng bộ với các hàm gọi lại (callbacks).


Sau khi thực hiện bài hướng dẫn này, ta sẽ hiểu được:


*  Cách Python đánh giá cú pháp decorator
*   Cách Python quyết định xem biến nào là cục bộ
*   Tại sao ta cần tới closures và cách chúng hoạt động
*  ```nonlocal``` được dùng để giải quyết vấn đề gì.

Sau khi trả lời các câu hỏi trên ta có được một hiểu biết nhất định để giải quyết các vấn đề sau:


*   Triển khai một decorator của riêng mình
*   Tìm hiểu các decorators trong thư viện chuẩn
*  Triển khai một decorator có tham sốsố

Chúng ta bắt đầu với các khái niệm căn bản về decoratorsors





## Decorator 101

Một decorator là một thứ có thể gọi (callable), chúng nhận vào một hàm khác (một hàm được trang trí (decoratored function)) và trả về hàm đó hay thay thế nó bằng một hàm khác hoặc một đối tượng có thể gọi khác. 

Nói cách khác, dòng code dưới đây

In [0]:
@decorate
def target():
  print('running target()')

Tương tự với các đoạn mã sau

In [0]:
def target():
  print('running target()')
  
target = decorate(target)

Kết quả của 2 đoạn mã trên là giống nhau; biến ```target``` không nhất thiết phải là hàm ```target``` ban đầu, nó có thể là bất cứ hàm nào được trả về bởi hàm ```decorate```.

Để chứng minh cái ta nói nãy giờ, thực hiện đoạn code sau

In [0]:
# Một decorator thường thay thế một hàm với một hàm khác
def deco(func):
  def inner():
    print('running inner()')
  return inner # decorator trả về một hàm bên trong nó

In [0]:
@deco
def target(): #hàm ```target``` là hàm được trang trí bởi ```deco```
  print('running target()')
  

In [4]:
# Gọi hàm được trang trí target sẽ trả về hàm inner, chứ không phải là hàm target ban đầu nữa
target()

running inner()


Bạn thấy ở trên khi ta gọi hàm ```target```, ta nhận được dòng "running inner", chứng tỏ rằng hàm được chạy thực chất là hàm ```inner```.

In [5]:
target

<function __main__.deco.<locals>.inner>

Biến target thực sự là một tham chiếu tới hàm ```inner```

Bạn có thể gọi một decorator như bất cứ callable nào, truyền cho nó một hàm khác. Điều này thỉnh thoảng hữu dụng khi ta thực hiện *metaprogramming* - thay đổi cư xử của chương trình tại thời điểm chạy.

Tổng kết lại: Decorator có tính năng thay thế một hàm được trang trí với một hàm khác. Điều quan trọng thứ hai đó là chúng được thực thi ngay sau khi module được nạp. Tôi sẽ giải thích điều đó ngay sau đây.

## Khi nào thì Python thực thi các decorators.

Một đặc tính của các decorators là chúng chạy ngay sau khi hàm được trang trí được định nghĩa. Xem xét ví dụ sau:


In [7]:
registry = [] # registry sẽ giữ các tham chiếu tới các hàm được trang trí bởi register

def register(func): # register nhận một hàm như là tham số
  print('running register (%s)' % func) # hiển thị tên hàm được trang trí
  registry.append(func) # cho tham chiếu tới hàm vào registry
  return func # trả về hàm y hệt như hàm nhận được

# f1 và f2 được trang trí bởi register
@register
def f1():
  print('running f1()')
  
@register
def f2():
  print('running f2()')
  
# f3 không được trang trí bởi register
def f3():
  print('running f3()')

running register (<function f1 at 0x7fbf7fc07f28>)
running register (<function f2 at 0x7fbf7fc07620>)


Ta có thể thấy ngay khi định nghĩa hàm thì các decorators đã chạy.

In [8]:
print('running main()')
print('registry -> ', registry)
f1()
f2()
f3()


running main()
registry ->  [<function f1 at 0x7fbf7fc07f28>, <function f2 at 0x7fbf7fc07620>]
running f1()
running f2()
running f3()
